$ \newcommand{\bra}[1]{\langle #1|} $
$ \newcommand{\ket}[1]{|#1\rangle} $

<h1>Automated parametrized solution</h1>

Here we implement an algorithm "Detecting a marked vertex" from the paper Quantum walk speedup of backtracking algorithms by Ashley Montanaro. We implement operations for a complete binary tree, we use two qubits per layer to use basis states that will correspond to specific vertices. Subsection "Generating layers - qubit basis state labels" is demonstrating the structure.

Remark - from the very beginning each gate is implemented with additional control qubit manually. For some reason automated functionality of Qiskit operator.contol() worked multiple times slower during the simulation.

<h2>Parameters</h2>

In [8]:
num_of_layers = 3
bits_of_precision = 6

<h2>Generating layers - qubit basis state labels</h2>

In [9]:
item_size = 2*num_of_layers
root = ''
for i in range(num_of_layers):
    root += '00'
layers = [[root]]

for cur_layer in range(1,num_of_layers+1):
    layers.append([])
    for i in (layers[cur_layer-1]):
        postfix = i[item_size-(cur_layer-1)*2:]
        prefix = ''
        for j in range(item_size-(cur_layer)*2):
            prefix += '0'
        layers[cur_layer].append(prefix+'01'+postfix)
        layers[cur_layer].append(prefix+'10'+postfix)
print(layers)

[['000000'], ['000001', '000010'], ['000101', '001001', '000110', '001010'], ['010101', '100101', '011001', '101001', '010110', '100110', '011010', '101010']]


<h2>Array for $\psi$</h2>

In this part we prepare states $\ket{\psi_x}$ for each vertex $x$.

In [10]:
psi = {}

<h2>Preparing root</h2>
<h3>$\ket{\psi_{00}}=\frac{1}{\sqrt{1+2*num\_of\_layers}}(\ket{00}+\sqrt{num\_of\_layers}\ket{01}+\sqrt{num\_of\_layers}\ket{10})$</h3>

In [11]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi, sqrt
qc = QuantumCircuit(2)

desired_vector = [
    1/sqrt(1+2*num_of_layers), # 00
    sqrt(num_of_layers)/sqrt(1+2*num_of_layers), # 01
    sqrt(num_of_layers)/sqrt(1+2*num_of_layers), # 10
    0 #11
]

q = QuantumRegister(2)
qc = QuantumCircuit(q)

qc.initialize(desired_vector, [q[0],q[1]])
qc2 = qc.decompose().decompose().decompose().decompose().decompose()

In [12]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi

psi[layers[0][0]] = QuantumCircuit(2*num_of_layers+1)
psi[layers[0][0]].cu3(qc2.data[1][0].params[0],0,0,0,1)
psi[layers[0][0]].cu3(qc2.data[3][0].params[0],0,0,0,2)
psi[layers[0][0]].ccx(0,2,1)
psi[layers[0][0]].cu3(qc2.data[5][0].params[0],0,0,0,1)
psi[layers[0][0]].ccx(0,2,1)

psi[layers[0][0]].draw()

q_0: ───────■───────────────■───────────■─────────■─────────■──
     ┌──────┴──────┐        │         ┌─┴─┐┌──────┴──────┐┌─┴─┐
q_1: ┤ U3(π/3,0,0) ├────────┼─────────┤ X ├┤ U3(π/3,0,0) ├┤ X ├
     └─────────────┘┌───────┴────────┐└─┬─┘└─────────────┘└─┬─┘
q_2: ───────────────┤ U3(1.4274,0,0) ├──■───────────────────■──
                    └────────────────┘                         
q_3: ──────────────────────────────────────────────────────────
                                                               
q_4: ──────────────────────────────────────────────────────────
                                                               
q_5: ──────────────────────────────────────────────────────────
                                                               
q_6: ──────────────────────────────────────────────────────────

<h2>States</h2>

In [13]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi

for m in range(1,num_of_layers):
    for k in layers[m]:
        psi[k] = QuantumCircuit(num_of_layers*2+1)
        for l in range(num_of_layers*2):
            if k[l] == '1':
                psi[k].cx(0,len(k)-l)
        psi[k].cu3(pi/4,0,0,0,2*m+1)
        psi[k].cu3(1.23095941734,0,0,0,2*m+2)
        psi[k].ccx(2*m+2,0,2*m+1)
        psi[k].cu3(pi/4,0,0,0,2*m+1)
        psi[k].ccx(2*m+2,0,2*m+1)
        
for k in layers[num_of_layers]:
    psi[k] = QuantumCircuit(num_of_layers*2+1)
    for l in range(len(k)):
        if k[l] == '1':
            psi[k].cx(0,len(k)-l)

<h2>Operators</h2>

In this part we prepare operators $D_x$ for each vertex $x$. Here $D_x = I - 2 \ket{\psi_x} \bra{\psi_x}$

In [14]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi

identity_minus_state_zero = QuantumCircuit(num_of_layers*2+1)

for i in range(num_of_layers*2):
    identity_minus_state_zero.cx(0,i+1)

control_states = []
for i in range(num_of_layers*2):
    control_states.append(i)

identity_minus_state_zero.h(num_of_layers*2)
identity_minus_state_zero.mct(control_states, num_of_layers*2)
identity_minus_state_zero.h(num_of_layers*2)

for i in range(num_of_layers*2):
    identity_minus_state_zero.cx(0,i+1)

d={}
for j in layers:
    for i in j: 
        d[i] = QuantumCircuit(num_of_layers*2+1)
        d[i] = d[i].compose(psi[i].inverse())
        d[i] = d[i].compose(identity_minus_state_zero)
        d[i] = d[i].compose(psi[i])

<h2>Implementation</h2>

Here we implement phase estimation algorithm. In steps where we prepare the transformation $R_BR_A$, for marked vertices $x$ we place identity operation instead of $D_x$.

We provide an array of indexes of marked elements for running the experiment. Array can be left empty for the case of no marked vertices.

In [15]:
marked=['010101','100101']

In [16]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi
from qiskit.circuit.library import QFT

r_a = QuantumCircuit(num_of_layers*2+1)

# root
r_a = r_a.compose(d[layers[0][0]])


for i in range(1,num_of_layers+1):
    if i%2 == 0:
        for j in layers[i]:
            if not j in marked:
                r_a = r_a.compose(d[j])
    
r_b = QuantumCircuit(num_of_layers*2+1)

for i in range(1,num_of_layers+1):
    if i%2 == 1:
        for j in layers[i]:
            if not j in marked:
                r_b = r_b.compose(d[j])

rbra = r_a
rbra = rbra.compose(r_b)

controlled_rbra = rbra
    
phase_estimation = QuantumCircuit(num_of_layers*2+bits_of_precision,bits_of_precision)
phase_estimation.h(range(bits_of_precision))

operation_qubits = []
for i in range(bits_of_precision,num_of_layers*2+bits_of_precision):
    operation_qubits = operation_qubits+[i]

qubits_to_appply = []
qubits_to_appply += [0]
qubits_to_appply += operation_qubits
for j in range(1):
    phase_estimation = phase_estimation.compose(controlled_rbra,qubits_to_appply)

for i in range(1, bits_of_precision):
    qubits_to_appply = []
    qubits_to_appply += [i]
    qubits_to_appply += operation_qubits
    controlled_rbra = controlled_rbra.compose(controlled_rbra)
    phase_estimation = phase_estimation.compose(controlled_rbra,qubits_to_appply)

phase_estimation.barrier()

phase_estimation = phase_estimation.compose(QFT(num_qubits=bits_of_precision, approximation_degree=0, do_swaps=False, inverse=True, insert_barriers=False, name='qft'),range(bits_of_precision))

phase_estimation.measure(range(bits_of_precision),range(bits_of_precision))
    
job = execute(phase_estimation,Aer.get_backend('qasm_simulator'),shots=10000)
counts = job.result().get_counts(phase_estimation)
print('marked: ['+', '.join(marked)+'], outcome: ')
n = ''
n = n.zfill(bits_of_precision)
result = counts.get(n, 0)
print(result/100,'%')
print(counts)

marked: [010101, 100101], outcome: 
55.5 %
{'001000': 297, '110111': 47, '000000': 5550, '110101': 71, '000010': 4, '101011': 296, '100001': 3, '111111': 10, '001010': 161, '001001': 41, '001101': 8, '100100': 282, '111100': 165, '101101': 8, '000100': 132, '110011': 10, '010111': 36, '111110': 3, '100010': 17, '001011': 84, '111000': 259, '001111': 1, '111010': 13, '100110': 67, '111001': 85, '001110': 1, '100101': 39, '111011': 34, '010101': 276, '101000': 127, '000001': 8, '101010': 130, '110110': 167, '110000': 3, '000111': 98, '011000': 149, '010100': 44, '011010': 71, '011110': 15, '010110': 135, '000011': 13, '101100': 43, '110100': 215, '011100': 265, '111101': 8, '100011': 17, '001100': 263, '000110': 7, '110001': 3, '101111': 4, '100000': 20, '010011': 8, '010000': 3, '011011': 34, '110010': 6, '101110': 7, '000101': 37, '100111': 17, '101001': 38, '010010': 7, '011101': 16, '011001': 13, '011111': 7, '010001': 2}


<h2>Running multiple experiments</h2>

We can also run multiple experiments with the code below.

In [17]:
experiments=[
    [],
    ['010101'],
    ['101010'],
    ['100101','010101'],
    ['011001','010101'],
    ['010110','010101']
]

In [18]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, Aer
from math import pi
from qiskit.circuit.library import QFT
from datetime import datetime
import time

for m in range(len(experiments)):
    marked = experiments[m]
    print('experiment started:',datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    r_a = QuantumCircuit(num_of_layers*2+1)

    # root
    r_a = r_a.compose(d[layers[0][0]])

    for i in range(1,num_of_layers+1):
        if i%2 == 0:
            for j in layers[i]:
                if not j in marked:
                    r_a = r_a.compose(d[j])
    
    r_b = QuantumCircuit(num_of_layers*2+1)

    for i in range(1,num_of_layers+1):
        if i%2 == 1:
            for j in layers[i]:
                if not j in marked:
                    r_b = r_b.compose(d[j])

    rbra = r_a
    rbra = rbra.compose(r_b)

    controlled_rbra = rbra
    
    phase_estimation = QuantumCircuit(num_of_layers*2+bits_of_precision,bits_of_precision)
    phase_estimation.h(range(bits_of_precision))

    operation_qubits = []
    for i in range(bits_of_precision,num_of_layers*2+bits_of_precision):
        operation_qubits = operation_qubits+[i]

    qubits_to_appply = []
    qubits_to_appply += [0]
    qubits_to_appply += operation_qubits
    for j in range(1):
        phase_estimation = phase_estimation.compose(controlled_rbra,qubits_to_appply)

    for i in range(1, bits_of_precision):
        qubits_to_appply = []
        qubits_to_appply += [i]
        qubits_to_appply += operation_qubits
        controlled_rbra = controlled_rbra.compose(controlled_rbra)
        phase_estimation = phase_estimation.compose(controlled_rbra,qubits_to_appply)

    phase_estimation.barrier()

    phase_estimation = phase_estimation.compose(QFT(num_qubits=bits_of_precision, approximation_degree=0, do_swaps=False, inverse=True, insert_barriers=False, name='qft'),range(bits_of_precision))

    phase_estimation.measure(range(bits_of_precision),range(bits_of_precision))
    
    job = execute(phase_estimation,Aer.get_backend('qasm_simulator'),shots=10000)
    counts = job.result().get_counts(phase_estimation)
    print('experiment ended:',datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
    print('marked: ['+', '.join(marked)+'], outcome: ')
    n = ''
    n = n.zfill(bits_of_precision)
    result = counts.get(n, 0)
    print(result/100,'%')
    print(counts)

experiment started: 2022-06-20 11:39:20
experiment ended: 2022-06-20 11:39:24
marked: [], outcome: 
0.12 %
{'011000': 2118, '000011': 3, '110100': 153, '101100': 807, '100000': 388, '101000': 2220, '011010': 149, '111011': 14, '001110': 31, '100101': 325, '110011': 7, '000100': 9, '101101': 117, '111110': 8, '100010': 35, '001011': 79, '010100': 794, '110010': 30, '000101': 18, '101110': 121, '000111': 1, '110000': 298, '001111': 16, '100110': 161, '111100': 13, '001101': 10, '100100': 190, '011101': 9, '011011': 322, '010000': 313, '111101': 2, '100011': 7, '001100': 157, '011001': 32, '111000': 80, '110111': 1, '000000': 12, '011100': 191, '010010': 132, '000010': 6, '101011': 29, '110101': 99, '010011': 109, '010101': 36, '101001': 8, '000001': 2, '110110': 13, '101010': 23, '010001': 17, '010110': 22, '100111': 51, '011110': 40, '100001': 8, '111111': 4, '001010': 18, '000110': 4, '110001': 6, '101111': 20, '011111': 6, '010111': 11, '001000': 91, '001001': 2, '111001': 2}
experime